In [69]:
import pandas as pd

train_path = 'train.csv'

data = pd.read_csv(train_path)
label = data.score
text = data.text
# print(score)


In [70]:
import nltk
# nltk.download()
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))

In [71]:
from bs4 import BeautifulSoup
import re

def comment_to_words(text):
    '''
    数据清理，去掉表情等非单词数据
    '''
    text = BeautifulSoup(text).get_text()

    words = re.sub("[^a-zA-Z]", " ", text)
    words = words.lower().split()
    words = [w for w in words if not w in stops]

    words = " ".join(words)

    return words


In [72]:
from tqdm import tqdm
n_comments = text.size
word_comments = []
for i in tqdm(range(n_comments)):
    word_comments.append(comment_to_words(text[i]))


100%|██████████| 7500/7500 [00:01<00:00, 5784.40it/s]


In [73]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, \
    preprocessor = None, stop_words = None, max_features = 10000)
train_features = vectorizer.fit_transform(word_comments)
train_features = train_features.toarray()

In [74]:
from sklearn.naive_bayes import MultinomialNB as MNB 
model = MNB()
model.fit(train_features, label)
from sklearn.model_selection import train_test_split
import numpy as np 
import sklearn

score = np.mean(sklearn.model_selection.cross_val_score(model, train_features, label, cv=50, scoring='roc_auc'))
print("20交叉验证得分：", score)


KeyboardInterrupt: 

In [ ]:
test_path = "test.csv"
test_data = pd.read_csv(test_path)
# print(test_data)
test_text = test_data.text
test_label = test_data.score


In [ ]:
print(test_label.index)

RangeIndex(start=0, stop=2500, step=1)


In [ ]:
test_word_comments = []
for i in tqdm(range(len(test_text))):
    test_word_comments.append(comment_to_words(test_text[i]))
# test_vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, \
    # preprocessor = None, stop_words = None, max_features = 10000)
test_features = vectorizer.transform(test_word_comments)
test_features = test_features.toarray()

result = model.predict(test_features)
output = pd.DataFrame(data={"score":result})
output.to_csv('result.csv', index = False, quoting=3)

100%|██████████| 2500/2500 [00:00<00:00, 5922.69it/s]


In [ ]:
# count = 0
# print(output)
r = output.score * test_label
    
print(r.sum()/2500)


0.3968
